In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('database').getOrCreate()

In [26]:
from pyspark.sql.functions import (col,expr,count,countDistinct,datediff,to_date,date_add,year,month,lag,lead,rank,max,min,round,
        sum,when,lit,desc,coalesce,abs,greatest,least,array,array_sort,substring, explode,explode_outer,posexplode,
        posexplode_outer,collect_list,array_intersect,concat_ws,sequence,
        unix_timestamp,rank,dense_rank,least,greatest,row_number,array_join,expr,trim,lower,array,sort_array,
        array_distinct,size,initcap,length,date_format,to_timestamp,concat,regexp_extract,length,regexp_replace,
        sum as spark_sum,sqrt,length,last,abs,avg,flatten,to_timestamp,array_contains,array_intersect,array_union,
        array_distinct,element_at,array_remove,row_number,ceil,floor,dayofweek,monotonically_increasing_id,last,first,
        asc_nulls_last,hours,hour,aggregate,weekofyear,dayofmonth,date_sub,array_distinct,percentile_approx,from_unixtime,
        lpad,rpad,array_intersect,array_except,dayofmonth,dayofyear,weekofyear,month,quarter,year,last_day,count,ceil
         )
from pyspark.sql.types import (StructField,StructType,
                    IntegerType,StringType,DateType,TimestampType )
from pyspark.sql import Window
from pyspark.sql import Row
#rlike,like,contains,array_join,collect_list,substring,array_size,cast,stack(to unpivot),LATERAL VIEW 
#Window.orderBy("id").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
#Window.partitionBy("category").orderBy("id").rangeBetween(Window.currentRow, 1)
#activity_df.withColumn('lead',lead('event_date',1).over(window_spec))
#withColumn('prev_num',lag(col('num'),1,0).over(window_spec))
#row_number(), rank, dense_rank(), sum(), count(), max(), min()
#, expr("sequence(start_year, end_year)"))
#.withColumn('yeareday',concat_ws('-',lit(year('start')),lit(12),lit(31)))
#cntcat_df.filter( cntcat_df['income'].between(*x) ).select(count('*').alias('cnt')

#### 1 10 Department Top Three Salaries H

In [27]:
department_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/1h_department.csv')
             .withColumnRenamed('id','dep_id')
             .withColumnRenamed('name','dep_name'))

employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/1h_employee.csv'))

In [30]:
department_df.printSchema()

root
 |-- dep_id: integer (nullable = true)
 |-- dep_name: string (nullable = true)



In [21]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- departmentId: integer (nullable = true)



In [42]:
window_spec = Window.partitionBy('departmentId').orderBy(col('salary').desc())
department_df.join(employee_df,department_df.dep_id == employee_df.departmentId,'inner' ).withColumn('rnk',dense_rank().over(window_spec)).show()


+------+--------+---+-----+------+------------+---+
|dep_id|dep_name| id| name|salary|departmentId|rnk|
+------+--------+---+-----+------+------------+---+
|     1|      IT|  4|  Max| 90000|           1|  1|
|     1|      IT|  1|  Joe| 85000|           1|  2|
|     1|      IT|  6|Randy| 85000|           1|  2|
|     1|      IT|  7| Will| 70000|           1|  3|
|     1|      IT|  5|Janet| 69000|           1|  4|
|     2|   Sales|  2|Henry| 80000|           2|  1|
|     2|   Sales|  3|  Sam| 60000|           2|  2|
+------+--------+---+-----+------+------------+---+



In [80]:
department_df.createOrReplaceTempView('department')
employee_df.createOrReplaceTempView('employee')

spark.sql('''
 with one as (select dep_id,dep_name,id,name,salary,departmentId from department 
                 inner join employee on department.dep_Id == employee.departmentId)
             select dep_id,dep_name,id,name,salary,rank() over(partition by departmentId order by salary desc) as rnk
                 from one
''').show()

+------+--------+---+-----+------+---+
|dep_id|dep_name| id| name|salary|rnk|
+------+--------+---+-----+------+---+
|     1|      IT|  4|  Max| 90000|  1|
|     1|      IT|  1|  Joe| 85000|  2|
|     1|      IT|  6|Randy| 85000|  2|
|     1|      IT|  7| Will| 70000|  4|
|     1|      IT|  5|Janet| 69000|  5|
|     2|   Sales|  2|Henry| 80000|  1|
|     2|   Sales|  3|  Sam| 60000|  2|
+------+--------+---+-----+------+---+



#### 2 13 Trips and Users H

In [28]:
trips_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/2h_trips.csv')
             .withColumnRenamed('client_id','cltId')
             .withColumnRenamed('driver_id','drId')
             .withColumnRenamed('city_id','ctyId')
             .withColumnRenamed('request_at','reqat'))

In [29]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/2h_users.csv'))

In [291]:
trips_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- cltId: integer (nullable = true)
 |-- drId: integer (nullable = true)
 |-- ctyId: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- reqat: string (nullable = true)



In [216]:
users_df.printSchema()

root
 |-- users_id: integer (nullable = true)
 |-- banned: string (nullable = true)
 |-- role: string (nullable = true)



In [172]:
x = users_df.filter(col('banned') == 'Yes').select('users_id').collect()
usrs = [x[i][0] for i in range(0,len(x))]
usrs

[2]

In [282]:
window_spec = Window.partitionBy('reqat').orderBy('reqat')
two_spec = Window.partitionBy('reqat','id')

In [284]:
trips_df.join(users_df,trips_df.id == users_df.users_id,'left_outer')\
.withColumn('totcnt',count('reqat').over(window_spec))\
.withColumn('cnt',when(col('status').like('cancelled%'),count('status').over(window_spec)).otherwise(0))\
.withColumn('bntclt',when((col('cltId').isin(usrs) | col('drId').isin(usrs)),count('id').over(two_spec)).otherwise(0))\
.withColumn('diff',when(col('bntclt')>0,(col('totcnt')-col('bntclt'))).otherwise(0))\
.withColumn('sm',sum('cnt').over(window_spec))\
.filter( (col('bntclt') != 0) & (col('diff') != 0) )\
.withColumn('CancellationRate',when(col('sm') == 0,col('cnt')/col('diff')).otherwise(round(col('bntclt')/col('diff'),2)))\
.select(col('reqat').alias('Day'),'CancellationRate').orderBy(col('reqat').asc()).show()

+----------+----------------+
|       Day|CancellationRate|
+----------+----------------+
|2013-10-01|            0.33|
|2013-10-02|             0.0|
|2013-10-03|             0.5|
+----------+----------------+



In [325]:
trips_df.createOrReplaceTempView('trips')
users_df.createOrReplaceTempView('users')

spark.sql('''
 with main as (select id,cltId,drId,ctyId,status,reqat,users_id,banned,
                 count(reqat) over(partition by reqat) as totcnt,
                 case when status like 'cancelled%' then count('status') over(partition by reqat) else 0 end cnt,
                 case when cltId in (2) then count('id') over(partition by reqat,id) else 0 end bntclt
                 from trips left join users on trips.id == users.users_id),
       one as (select *,sum(cnt) over(partition by reqat) as sm,
                 case when bntclt > 0 then (totcnt - bntclt) else 0 end as diff  from main)
               select reqat,case when sm == 0 then round(sm/diff,2) else round(bntclt/diff,2) end CancellationRate 
                 from one  where bntclt != 0 and diff != 0 order by reqat asc
        ''').show()

+----------+----------------+
|     reqat|CancellationRate|
+----------+----------------+
|2013-10-01|            0.33|
|2013-10-02|             0.0|
|2013-10-03|             0.5|
+----------+----------------+



#### 3 18 Median Employee Salary H

In [30]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/3h_employee.csv'))

In [4]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- salary: integer (nullable = true)



In [27]:
window_spec = Window.partitionBy('company').orderBy(col('salary').asc())
cnt_spec = Window.partitionBy('company')

In [130]:
window_spec = Window.partitionBy('company').orderBy(col('salary').asc())
cnt_spec = Window.partitionBy('company')
mx_spec =  Window.partitionBy('company','num').orderBy(col('id').asc())

employee_df.withColumn('rnk',rank().over(window_spec)).withColumn('cnt',count('company').over(cnt_spec))\
.withColumn('mx',max('rnk').over(cnt_spec))\
.withColumn('num',when( (col('mx')%2 == 0) & ( ((col('mx') - col('mx')/2) == (col('rnk')))\
                                           | ( ((col('mx') - col('mx')/2)+1 == (col('rnk'))) ) ),col('rnk') )\
                 .when( (col('mx')%2 == 1) & ( ceil(col('mx') - col('mx')/2) == col('rnk')),ceil(col('mx') - col('mx')/2) ).otherwise(0))\
.withColumn('ids',when( (col('num') != 0) & (col('mx')%2 == 1), min('id').over(mx_spec)) \
                 .when( (col('num') != 0),col('id')))\
.filter(col('id') == col('ids')).select('id','company','salary').orderBy('company').show()


+---+-------+------+
| id|company|salary|
+---+-------+------+
|  6|      A|   513|
|  5|      A|   451|
| 12|      B|   234|
|  9|      B|  1154|
| 14|      C|  2645|
+---+-------+------+



In [156]:
employee_df.createOrReplaceTempView('employee')


spark.sql('''
 with main as (select id,company,salary,rank() over(partition by company order by salary asc ) as rnk from employee),
       one as (select *,max(rnk) over(partition by company) as mx from main),
       two as (select *,case when mx%2 == 0 and ( (mx - ceil(mx/2) == rnk) or (mx - ceil(mx/2)+1 ==rnk ) ) then rnk 
                             when mx%2 == 1 and (  mx - floor(mx/2) == rnk) then rnk 
                        else 0 end num from one),
     three as (select *,case when num != 0 and mx%2 ==1 then min(id) over(partition by company,num)
                             when num != 0 then id else 0 end ids from two)
               select id,company,salary from three where id == ids order by company
        ''').show()

+---+-------+------+
| id|company|salary|
+---+-------+------+
|  6|      A|   513|
|  5|      A|   451|
| 12|      B|   234|
|  9|      B|  1154|
| 14|      C|  2645|
+---+-------+------+



#### 4 20 Find Median Given Frequency of Numbers  H

In [31]:
numbers_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/4h_numbers.csv'))

In [161]:
numbers_df.printSchema()

root
 |-- num: integer (nullable = true)
 |-- frequency: integer (nullable = true)



In [191]:
dic = {i['num']:i['frequency'] for i in numbers_df.collect()}

In [234]:
lst =  sorted([i for i,j in dic.items() for k in range(0,j)])

In [256]:
str = ''
if len(lst)%2 == 0:
    even = int(len(lst) - (len(lst)/2))
    str =  int((lst[even]+lst[even-1])/2)
elif len(lst)%2 == 1:
    odd = int(len(lst)/2)
    str = odd
print(f'Median of the given Numbers:{str}')

Median of the given Numbers:0


#### 5 28 Find Cumulative Salary of an Employee H

In [32]:
employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/5h_employee.csv'))

In [169]:
employee_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [172]:
window_spec = Window.partitionBy('id').orderBy(col('id').asc(),col('month').desc())

df = employee_df.withColumn('diff',col('month') - lead('month',1,0).over(window_spec))\
.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') != 1).select('id','month','salary','diff').collect()

In [173]:
def missing_months():
    lst =  []
    lst2 = []
    columns = ["id", "month", "salary"]
    for i in range(0,len(df)):   
        if df[i]['diff'] > 1:
            lst.append( (df[i][0],df[i][1],df[i][2],df[i][-1]))

    for i in range(0,len(lst)):
        for j in range(1,lst[i][3]):
            out = (lst[i][0],lst[i][1]-j,0)
            lst2.append(out)

    return  spark.createDataFrame(lst2, columns)

In [ ]:
window_spec = Window.partitionBy('id').orderBy(col('id').asc(),col('month').desc())
bt_spec = Window.partitionBy('id').orderBy(col('id').asc(),col('month').desc()).rowsBetween(0,2)

employee_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') != 1)\
.select('id','month','salary').union(missing_months()).withColumn('cumulative_sum',sum('salary').over(bt_spec))\
.filter(col('salary') != 0).select('id','month',col('cumulative_sum').alias('salary') ).orderBy(col('id').asc(),col('month').desc()).show()

In [173]:
employee_df.createOrReplaceTempView('employee')
missing_months().createOrReplaceTempView('missing_months')

spark.sql('''
 with main as (select id,month,salary,rank() over(partition by id order by id asc,month desc) as rnk from employee),
       one as ((select id,month,salary from main where rnk != 1) UNION (select * from missing_months)),
       two as (select id,month,salary,sum(salary) over(partition by id order by id asc,month desc ROWS BETWEEN 0 PRECEDING AND 2 FOLLOWING) as cumulative_sum from one)
               select id,month,cumulative_sum as salary from two where salary != 0 order by id asc,month desc
          ''').show()

+---+-----+------+
| id|month|salary|
+---+-----+------+
|  1|    7|    90|
|  1|    4|   130|
|  1|    3|    90|
|  1|    2|    50|
|  1|    1|    20|
|  2|    1|    20|
|  3|    3|   100|
|  3|    2|    40|
+---+-----+------+



#### 6 36 Human Traffic of Stadium H

In [33]:
stadium_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/6h_stadium.csv'))

In [176]:
stadium_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- visit_date: string (nullable = true)
 |-- people: integer (nullable = true)



In [283]:
std_df = stadium_df.collect()

def consecutive_id():
    lst = []
    for i in range(0,len(std_df)):
        x = (std_df[i][0],std_df[i][2])
        lst.append(x)

    id_lst = set()
    in_lst = []
    cnt = 1
    for i in range(0,len(lst)):
        if lst[i][1] >= 100:
            in_lst.append(lst[i][0])
            cnt +=1
            if cnt >= 3:
                id_lst.update(in_lst)
        else:
            cnt = 0
            in_lst = []
    return list(id_lst)

def to_dataframe():
    return spark.createDataFrame([ (i,) for i in consecutive_id()],['con_id'] )

In [284]:
stadium_df.filter(col('id').isin(consecutive_id())).show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



In [287]:
stadium_df.createOrReplaceTempView('stadium')
to_dataframe().createOrReplaceTempView(('to_dataframe'))

spark.sql('''
          select id,visit_date,people 
              from stadium join to_dataframe on stadium.id == to_dataframe.con_id order by id asc
         ''').show()

+---+----------+------+
| id|visit_date|people|
+---+----------+------+
|  5|2017-01-05|   145|
|  6|2017-01-06|  1455|
|  7|2017-01-07|   199|
|  8|2017-01-09|   188|
+---+----------+------+



#### 7 45 Average Salary: Departments VS Company H

In [34]:
salary_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/7h_salary.csv'))

employee_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/7h_employee.csv')
             .withColumnRenamed('employee_id','emp_id'))

In [298]:
salary_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- employee_id: integer (nullable = true)
 |-- amount: integer (nullable = true)
 |-- pay_date: string (nullable = true)



In [301]:
employee_df.printSchema()

root
 |-- emp_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)



In [349]:
window_spec = Window.partitionBy('pay_date')
avg_spec = Window.partitionBy('pay_date','department_id')

salary_df.join(employee_df,salary_df.employee_id == employee_df.emp_id,'inner' ).orderBy('department_id','emp_id')\
.withColumn("pay_date",to_date("pay_date", "yyyy/MM/dd")).withColumn('pay_date',date_format('pay_date','yyyy-MM'))\
.withColumn('sm',round(sum('amount').over(window_spec)/count('amount').over(window_spec),2))\
.withColumn('avg',round(sum('amount').over(avg_spec)/count('amount').over(avg_spec),2))\
.withColumn('comparison',when((col('avg') > col('sm')), lit('higher'))
                        .when((col('avg') < col('sm')), lit('lower'))
                        .when((col('avg') == col('sm')), lit('same')))\
.select('pay_date','department_id','comparison').distinct().orderBy('department_id','pay_date').show()

+--------+-------------+----------+
|pay_date|department_id|comparison|
+--------+-------------+----------+
| 2017-02|            1|      same|
| 2017-03|            1|    higher|
| 2017-02|            2|      same|
| 2017-03|            2|     lower|
+--------+-------------+----------+



In [380]:
salary_df.createOrReplaceTempView('salary')
employee_df.createOrReplaceTempView('employee')


spark.sql('''
 with main as (select id,employee_id,amount,date_format(to_date(pay_date, "yyyy/MM/dd"),'yyyy-MM') as pay_date,emp_id,department_id from salary
                    join employee on employee_id == emp_id),
       one as (select pay_date,amount,emp_id,department_id,
                  round(sum(amount) over(partition by pay_date)/count(amount) over(partition by pay_date),2)  as sm,
                  round(sum(amount) over(partition by pay_date,department_id)/count(amount) over(partition by pay_date,department_id),2)  as avg from main),
       two as (select pay_date,department_id,case when avg > sm then 'higher'
                                                  when avg < sm then 'lower'
                                                  when avg == sm then 'same' end comparison  from one)
              select distinct pay_date,department_id,comparison from two order by department_id,pay_date
        ''').show()

+--------+-------------+----------+
|pay_date|department_id|comparison|
+--------+-------------+----------+
| 2017-02|            1|      same|
| 2017-03|            1|    higher|
| 2017-02|            2|      same|
| 2017-03|            2|     lower|
+--------+-------------+----------+



#### 8 46 Students Report By Geography H

In [35]:
student_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/8h_student.csv'))

In [4]:
student_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- continent: string (nullable = true)



In [71]:
pivot_df = student_df.groupBy().pivot("continent").agg(collect_list(col('name')))

In [103]:
pivot_df = student_df.groupBy().pivot("continent").agg(collect_list(col('name')))

america_df = pivot_df.select(posexplode("America")).select('pos',col('col').alias('America'))
asia_df = pivot_df.select(posexplode("Asia")).select(col('pos').alias('pos1'),col('col').alias('Asia'))
europe_df = pivot_df.select(posexplode("Europe")).select(col('pos').alias('pos1'),col('col').alias('Europe'))

america_df.join(asia_df,america_df.pos == asia_df.pos1,'outer').join(europe_df, america_df.pos == europe_df.pos1,'outer')\
.orderBy(col('pos').asc()).drop('pos1','pos').show()

+-------+----+------+
|America|Asia|Europe|
+-------+----+------+
|   Jane|  Xi|Pascal|
|   Jack|null|  null|
+-------+----+------+



#### 9 62 Game Play Analysis V H

In [36]:
activity_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/9h_activity.csv'))

In [117]:
activity_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- event_date: string (nullable = true)
 |-- games_played: integer (nullable = true)



In [231]:
window_spec = Window.partitionBy('player_id').orderBy(col('player_id').asc(),col('event_date').asc())
cnt_spc =     Window.partitionBy('event_date')

one = activity_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 1).select('player_id','event_date')
two = activity_df.withColumn('rnk',rank().over(window_spec)).filter(col('rnk') == 2)\
                     .select(col('player_id').alias('id'),col('event_date').alias('date'))

one.join(two,one.player_id == two.id,'outer').withColumn('cnt',count('event_date').over(cnt_spc))\
.withColumn('diff',ceil(round((unix_timestamp(to_date(col('date'),'yyyy-MM-dd')) - unix_timestamp(to_date(col('event_date'),'yyyy-MM-dd')))/86400,0)) )\
.withColumn('agg',when( (col('cnt') >= 2) & (col('diff') == 1),count('event_date').over(cnt_spc) ))\
.withColumn('Day1_retention',when( (col('cnt') >= 2) & (col('diff') == 1),col('diff')/col('agg'))
                 .when( (col('cnt') == 1), 0/col('cnt'))).filter(col('Day1_retention').isNotNull())\
                 .select(col('event_date').alias('install_dt'),col('cnt').alias('installs'),'Day1_retention').orderBy(col('install_dt').asc()).show()

+----------+--------+--------------+
|install_dt|installs|Day1_retention|
+----------+--------+--------------+
|2016-03-01|       2|           0.5|
|2017-06-25|       1|           0.0|
+----------+--------+--------------+



In [323]:
one.createOrReplaceTempView('one')
two.createOrReplaceTempView('two')

def out():
    x = spark.sql('''
     with main as (select player_id,event_date,id,date,count(event_date) over(partition by event_date) as cnt,
                       ceil(round((unix_timestamp(to_date(date,'yyyy-MM-dd')) - unix_timestamp(to_date(event_date,'yyyy-MM-dd')))/86400,0)) as diff
                       from one full outer join two on player_id == id),
           one as (select * , case when (cnt >= 2) and (diff == 1) then count(event_date) over(partition by event_date) end as agg  from main),
           two as (select *,case when ((cnt >= 2) and (diff == 1)) then diff/agg
                                 when  cnt ==1 then 0/cnt end Day1_retention from one)
                   select event_date as install_dt,cnt as installs,Day1_retention from two 
                      where Day1_retention is not null order by event_date asc
            ''')
    return x

out().show()

+----------+--------+--------------+
|install_dt|installs|Day1_retention|
+----------+--------+--------------+
|2016-03-01|       2|           0.5|
|2017-06-25|       1|           0.0|
+----------+--------+--------------+



#### 10 68 User Purchase Platform H

In [37]:
spending_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/10h_spending.csv'))

In [4]:
spending_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- spend_date: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- amount: integer (nullable = true)



In [61]:
window_spec = Window.partitionBy('user_id','spend_date')
one_spec = Window.partitionBy('user_id','spend_date','platform')
dt_spec = Window.partitionBy('spend_date').orderBy(col('spend_date').asc())

df = spending_df.withColumn('cnt',count('user_id').over(window_spec)).withColumn('cntplt',count('user_id').over(one_spec))\
.withColumn('diff',col('cnt') - col('cntplt'))\
.withColumn('plt',when( col('cnt') > 1, lit('both')).otherwise(col('platform')))\
.withColumn('sm',when(col('cnt') > 1,sum('amount').over(window_spec)).otherwise(col('amount')))\
.select('spend_date','plt','sm','cntplt').distinct().orderBy('spend_date').collect()

from collections import Counter

def cnt():
    columns = ['spend_date','plt','sm','cntplt']
    lst = []
    for i in range(0,len(df)):
        lst.append(df[i]['spend_date'])
        
    lst2 = []
    date_counts = Counter(lst)
    for i,j in date_counts.items():
        if j < 3:
            x =  (i,'both',0,0)
            lst2.append(x)
    return spark.createDataFrame(lst2, columns)

spending_df.withColumn('cnt',count('user_id').over(window_spec)).withColumn('cntplt',count('user_id').over(one_spec))\
.withColumn('diff',col('cnt') - col('cntplt'))\
.withColumn('plt',when( col('cnt') > 1, lit('both')).otherwise(col('platform')))\
.withColumn('sm',when(col('cnt') > 1,sum('amount').over(window_spec)).otherwise(col('amount')))\
.select('spend_date','plt','sm','cntplt').distinct()\
.select('spend_date',col('plt').alias('platform'),col('sm').alias('total_amount'),col('cntplt').alias('total_users'))\
.orderBy('spend_date','total_amount').union(cnt()).show()


+----------+--------+------------+-----------+
|spend_date|platform|total_amount|total_users|
+----------+--------+------------+-----------+
|2019-07-01|  mobile|         100|          1|
|2019-07-01| desktop|         100|          1|
|2019-07-01|    both|         200|          1|
|2019-07-02|  mobile|         100|          1|
|2019-07-02| desktop|         100|          1|
|2019-07-02|    both|           0|          0|
+----------+--------+------------+-----------+



#### 11 75 Market Analysis II H

In [38]:
users_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/11h_users.csv'))

In [39]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/11h_orders.csv'))

In [40]:
items_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/11h_items.csv'))

In [65]:
users_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- join_date: string (nullable = true)
 |-- favorite_brand: string (nullable = true)



In [66]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- buyer_id: integer (nullable = true)
 |-- seller_id: integer (nullable = true)



In [67]:
items_df.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- item_brand: string (nullable = true)



In [98]:
window_spec = Window.partitionBy('user_id').orderBy('user_id',col('brand').asc()).rowsBetween(0,2)

users_df.join(orders_df,users_df.user_id == orders_df.seller_id,'left')\
.join(items_df,orders_df.item_id == items_df.item_id,'left' )\
.withColumn('brand',when( col('item_brand').isNull(),lit('no'))\
                   .when( col('item_brand') == col('favorite_brand'),lit('yes')).otherwise('no'))\
.withColumn('cnt',count('brand').over(window_spec)).filter(col('cnt') == 1)\
.select(col('user_id').alias('seller_id'),col('brand').alias('2nd_item_fav_brand')).orderBy('seller_id').show()

+---------+------------------+
|seller_id|2nd_item_fav_brand|
+---------+------------------+
|        1|                no|
|        2|               yes|
|        3|               yes|
|        4|                no|
+---------+------------------+



In [109]:
users_df.createOrReplaceTempView('users')
orders_df.createOrReplaceTempView('orders')
items_df.createOrReplaceTempView('items')

spark.sql('''
 with main as (select user_id,favorite_brand,orders.item_id,seller_id,item_brand from users
                  left join orders on users.user_id == orders.seller_id
                  left join items  on items.item_id == orders.item_id),
       one as (select user_id,favorite_brand,item_id,seller_id,item_brand,
                  case when item_brand is null then 'no'
                       when item_brand == favorite_brand then 'yes' else 'no' end brand from main),
       two as (select * , count(brand) over(partition by user_id order by user_id,brand asc ROWS BETWEEN 0 PRECEDING AND 2 FOLLOWING) as cnt from one)
               select user_id as seller_id,brand as 2nd_item_fav_brand from two where cnt == 1 order by  user_id              
        ''').show()

+---------+------------------+
|seller_id|2nd_item_fav_brand|
+---------+------------------+
|        1|                no|
|        2|               yes|
|        3|               yes|
|        4|                no|
+---------+------------------+



#### 12 81 Tournament Winners H

In [41]:
players_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/12h_payers.csv'))

In [42]:
matches_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/12h_matches.csv'))

In [6]:
players_df.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- group_id: integer (nullable = true)



In [7]:
matches_df.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- first_player: integer (nullable = true)
 |-- second_player: integer (nullable = true)
 |-- first_score: integer (nullable = true)
 |-- second_score: integer (nullable = true)



In [25]:
window_spec = Window.partitionBy('group_id','player').orderBy('group_id','player')
mx_spec = Window.partitionBy('group_id')

x = players_df.join(matches_df,players_df.player_id == matches_df.first_player,'inner')\
.select('group_id','match_id',col('second_player').alias('player'),col('second_score').alias('score'))

players_df.join(matches_df,players_df.player_id == matches_df.first_player,'inner')\
.select('group_id','match_id',col('first_player').alias('player'),col('first_score').alias('score'))\
.unionAll(x).withColumn('sm',sum('score').over(window_spec)).withColumn('mx',max('sm').over(mx_spec))\
.withColumn('diff',when( col('sm') == col('mx'),1).otherwise(0)).filter(col('diff') ==1 )\
.withColumn('mn',min('player').over(mx_spec)).select('group_id',col('mn').alias('player_id')).distinct().orderBy('group_id').show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+



In [55]:
players_df.createOrReplaceTempView('players')
matches_df.createOrReplaceTempView('matches')

spark.sql('''
 with main as (select * from players join matches on players.player_id == matches.first_player),
       one as ((select group_id,first_player as player,first_score as score from main)
                    UNION ALL (select group_id,second_player as player,second_score as score from main)),
       two as (select group_id,player,score,sum(score) over(partition by group_id,player order by group_id,player) as sm from one),
     three as (select group_id,player,score,sm,max(sm) over(partition by group_id) mx from two),
      four as (select group_id,player,score,sm,mx,diff,min(player) over(partition by group_id ) mn from 
                    (select group_id,player,score,sm,mx,case when sm == mx then 1 else 0 end diff from three)
                    where diff == 1)
                select distinct group_id,mn as player_id from four order by group_id
          ''').show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+



#### 13 86 Report Contiguous Dates H

In [43]:
failed_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/13h_failed.csv'))

In [44]:
succeeded_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/13h_succeeded.csv'))

In [58]:
failed_df.printSchema()

root
 |-- fail_date: string (nullable = true)



In [59]:
succeeded_df.printSchema()

root
 |-- success_date: string (nullable = true)



In [189]:
window_spec = Window.orderBy('fail_date','state')
sm_spec = Window.partitionBy('state').orderBy('cnt').rangeBetween(-1,1)

x = failed_df.withColumn('state',lit('fail')).union(succeeded_df.withColumn('state',lit('succeeded') ))\
.withColumn('sbdate',substring(col('fail_date'),1,4)).filter(col('sbdate') != 2018 )\
.withColumn('cnt',count('fail_date').over(window_spec)).withColumn('mx',max('cnt').over(sm_spec))\
.withColumn('mn',min('cnt').over(sm_spec))\
.withColumn('start_date',when( col('cnt') == col('mn'),col('fail_date')))\
.withColumn('end_date',when(col('cnt') == col('mx'),col('fail_date') ))\
.withColumn('diff',col('mx') -  col('mn')).filter(col('diff') <= 1)

one = x.filter(~col('start_date').isNull()).select('state','start_date','diff')
two = x.filter(~col('end_date').isNull()).select( col('state').alias('st'),'end_date',col('diff').alias('di'))

one.join(two, (one.diff == two.di) & (one.state == two.st) ,'inner')\
.select(col('state').alias('period_state'),'start_date','end_date').orderBy('start_date').show()

+------------+----------+----------+
|period_state|start_date|  end_date|
+------------+----------+----------+
|   succeeded|2019-01-01|2019-01-03|
|        fail|2019-01-04|2019-01-05|
|   succeeded|2019-01-06|2019-01-06|
+------------+----------+----------+



24/11/15 18:04:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/15 18:04:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


#### 14 99 Number of Transactions per Visit H

In [45]:
visits_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/14h_visits.csv')
             .withColumnRenamed('user_id','id'))

In [46]:
transactions_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/14h_transactions.csv'))

In [196]:
visits_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- visit_date: string (nullable = true)



In [193]:
 transactions_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- amount: integer (nullable = true)



In [280]:
window_spec = Window.partitionBy('user_id','transaction_date').orderBy('user_id','transaction_date')
cnt_spec = Window.partitionBy('t_count')

x = visits_df.join(transactions_df,(visits_df.visit_date == transactions_df.transaction_date) 
               & (visits_df.id == transactions_df.user_id),'left')\
.withColumn('cnt',count('id').over(window_spec))\
.withColumn('t_count',when(col('amount').isNull(),lit(0))\
                     .when(col('cnt') == 1,lit(1))\
                     .when(col('cnt') == 2,lit(2))
                     .when(col('cnt') == 3,lit(3)))\
.withColumn('rnk',rank().over(window_spec))\
.withColumn('bt_count',count('t_count').over(cnt_spec))\
.withColumn('v_count',when( col('t_count') == col('bt_count'),col('rnk'))\
                     .when( col('t_count') != col('bt_count'),col('bt_count')))\
.select('t_count','v_count').distinct().orderBy('t_count').collect()

def miss_val():
    lst = []
    colums = ['t_count','v_count']
    for i in range(0,len(x)-1):
        if ((x[i+1]['t_count']) - (x[i]['t_count'])) > 1:
            out = x[i]['t_count']+1,0
            lst.append(out)
    return spark.createDataFrame(lst,colums)

spark.createDataFrame(x,['t_count','v_count']).union(miss_val())\
.select(col('t_count').alias('transactions_count'),col('v_count').alias('visits_count')).orderBy('transactions_count').show()

+------------------+------------+
|transactions_count|visits_count|
+------------------+------------+
|                 0|           4|
|                 1|           5|
|                 2|           0|
|                 3|           1|
+------------------+------------+



#### 15 104  Get the Second Most Recent Activity H

In [47]:
useractivity_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/15h_useractivity.csv'))

In [5]:
useractivity_df.printSchema()

root
 |-- username: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- startDate: string (nullable = true)
 |-- endDate: string (nullable = true)



In [21]:
window_spec = Window.partitionBy('username').orderBy('username','startDate')
lag_spec =  Window.partitionBy('username')

useractivity_df.withColumn('rnk',rank().over(window_spec)).withColumn('cnt',count('username').over(lag_spec))\
.withColumn('fil',when( (col('rnk') == 1) &  (col('cnt') == 1 ),lit(1))
                 .when( (col('rnk') == 2) &  (col('cnt') >= 2 ),lit(1)).otherwise(lit(0)))\
.filter(col('fil') == 1).select('username','activity','startDate','endDate').orderBy('username').show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice| Dancing|2020-02-21|2020-02-23|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+



In [32]:
useractivity_df.createOrReplaceTempView('useractivity')

spark.sql('''
 with main as (select username,activity,startDate,endDate,rank() over(partition By username order by username,startDate) as rnk,
                    count(username) over(partition By username) as cnt from useractivity),
       one as (select * ,case when rnk == 1 and cnt == 1 then 1
                            when rnk == 2 and cnt >=  2 then 1 else 0 end fil from main)
               select username,activity,startDate,endDate from one where fil == 1 order by username
       ''').show()

+--------+--------+----------+----------+
|username|activity| startDate|   endDate|
+--------+--------+----------+----------+
|   Alice| Dancing|2020-02-21|2020-02-23|
|     Bob|  Travel|2020-02-11|2020-02-18|
+--------+--------+----------+----------+



#### 16 106 Total Sales Amount by Year H

In [48]:
schema_info = StructType([StructField('product_id',IntegerType()),
                         StructField('period_start',DateType()),
                         StructField('period_end',DateType()),
                         StructField('average_daily_sales',IntegerType())] )

In [49]:
product_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/16h_product.csv'))

In [50]:

sales_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .schema(schema_info)
             .format('csv')
             .load('../../data/database/16h_sales.csv')
             .withColumn('period_start',unix_timestamp('period_start'))
             .withColumn('period_end',unix_timestamp('period_end'))
             .withColumnRenamed('product_id','id'))

In [72]:
product_df.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)



In [73]:
sales_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- period_start: long (nullable = true)
 |-- period_end: long (nullable = true)
 |-- average_daily_sales: integer (nullable = true)



In [231]:
sales_df.withColumn('diff',(col('period_end') - col('period_start'))/86400  )\
.withColumn('start',from_unixtime(col('period_start'),'yyyy-MM-dd'))\
.withColumn('end',  from_unixtime(col('period_end'),'yyyy-MM-dd'))\
.select('id',col('average_daily_sales').alias('sales'),'start','end')\
.withColumn("start_year", year("start")) \
    .withColumn("end_year", year("end")) \
    .withColumn("years", expr("sequence(start_year, end_year)"))\
    .selectExpr("id", "explode(years) as report_year", "start", "end", "sales")\
.withColumn("start_date",greatest(to_date(col('start'),'yyyy-MM-dd'),to_date(expr("concat(report_year, '-01-01')"),'yyyy-MM-dd')))\
.withColumn("end_date",least(to_date(col('end'),'yyyy-MM-dd'),to_date(expr("concat(report_year, '-12-31')"),'yyyy-MM-dd')))\
.withColumn("days", datediff(col("end_date"), col("start_date")) + lit(1))\
.withColumn("total_amount", col("days") * col("sales"))\
.join(product_df,product_df.product_id == sales_df.id,'inner')\
.select('product_id','product_name','report_year','total_amount').orderBy('product_id','report_year')\
.show()

+----------+------------+-----------+------------+
|product_id|product_name|report_year|total_amount|
+----------+------------+-----------+------------+
|         1|    LC Phone|       2019|        3500|
|         2|  LC T-Shirt|       2018|         310|
|         2|  LC T-Shirt|       2019|        3650|
|         2|  LC T-Shirt|       2020|          10|
|         3| LC Keychain|       2019|          31|
|         3| LC Keychain|       2020|          31|
+----------+------------+-----------+------------+



#### 17 110 Find the Quiet Students in All Exams H

In [51]:
exam_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/17h_exam.csv'))

In [52]:
student_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/17h_student.csv')
             .withColumnRenamed('student_id','id'))

In [268]:
exam_df.printSchema()

root
 |-- exam_id: integer (nullable = true)
 |-- student_id: integer (nullable = true)
 |-- score: integer (nullable = true)



In [269]:
student_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- student_name: string (nullable = true)



In [271]:
window_spc = Window.partitionBy('exam_id')

mx = exam_df.withColumn('mx',max('score').over(window_spc)).filter(col('score') == col('mx')).select('student_id')
mn = exam_df.withColumn('mn',min('score').over(window_spc)).filter(col('score') == col('mn')).select('student_id')

min_max = set([mx.union(mn).collect()[i]['student_id'] for i in range(0,len(mx.union(mn).collect()))] )

min_max
exam_df.filter(~col('student_id').isin(min_max)).join(student_df,student_df.id == exam_df.student_id,'inner')\
.select('student_id','student_name').distinct().show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         2|        Jade|
+----------+------------+



#### 18 118 Sales by Day of the Week H

In [53]:
orders_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/18h_orders.csv'))

In [54]:
items_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/18h_items.csv')
             .withColumnRenamed('item_id','id'))

In [282]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [283]:
items_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_category: string (nullable = true)



In [301]:
window_spec = Window.partitionBy('days','item_category')

items_df.join(orders_df,orders_df.item_id == items_df.id,'outer').select('item_category','order_date','quantity')\
.withColumn('day',dayofweek('order_date')).withColumn('days',when(col('day') == 1,lit('Sunday'))
                                                            .when(col('day') == 2,lit('Monday'))
                                                            .when(col('day') == 3,lit('Tuesday'))
                                                            .when(col('day') == 4,lit('Wednesday'))
                                                            .when(col('day') == 5,lit('Thursday'))                                                            .when(col('day') == 2,lit('Monday'))
                                                            .when(col('day') == 6,lit('Friday'))
                                                            .when(col('day') == 7,lit('Saturday')) )\
.withColumn('qty',sum('quantity').over(window_spec)).select('item_category','days','qty').distinct()\
.groupBy('item_category').pivot('days',["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]).sum('qty')\
.fillna(0).withColumnRenamed('item_category','category').orderBy('category').show()
                               

+--------+------+-------+---------+--------+------+--------+------+
|category|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|
+--------+------+-------+---------+--------+------+--------+------+
|    Book|    20|      5|        0|       0|    10|       0|     0|
| Glasses|     0|      0|        0|       0|     5|       0|     0|
|   Phone|     0|      0|        5|       1|     0|       0|    10|
| T-Shirt|     0|      0|        0|       0|     0|       0|     0|
+--------+------+-------+---------+--------+------+--------+------+



#### 19 250 CEO Subordinate Hierarchy H

In [55]:
employees_df = (spark.read
             .option('header',True)
             .option('inferSchema',True)
             .format('csv')
             .load('../../data/database/19h_employees.csv'))

In [4]:
employees_df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- manager_id: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [76]:
employees_df.alias('e').join(employees_df.alias('m'),col('e.employee_id') == col('m.manager_id'),'inner')\
.select(col('m.employee_id').alias('subordinate_id'),col('m.employee_name').alias('subordinate_name'),
       when( col('e.manager_id').isNull(),lit(1))
      .when( col('e.manager_id') == 1, lit(2))
      .when( col('e.manager_id') == 2, lit(3)).alias('hierarchy_level'),
      (col('m.salary') - lit(150000) ).alias('salary')).show()

+--------------+----------------+---------------+------+
|subordinate_id|subordinate_name|hierarchy_level|salary|
+--------------+----------------+---------------+------+
|             2|             Bob|              1|-30000|
|             3|         Charlie|              1|-40000|
|             4|           David|              2|-45000|
|             5|             Eve|              2|-50000|
|             6|           Frank|              2|-55000|
|             7|           Grace|              2|-52000|
|             8|           Helen|              3|-60000|
+--------------+----------------+---------------+------+

